## Imports

In [29]:
import pandas as pd
import numpy as np

## Reading in transaction data

In [30]:
transactions = pd.read_csv("C:\\Users\\shani\\PycharmProjects\\preppindata\\Transactions.csv")

In [31]:
transactions.head()

,Transaction ID,Account Number,Sort Code,Bank
0,3888,62230725,95-98-82,Data Source Bank
1,4746,83172326,42-86-38,Barclays Bank
2,5404,34302539,53-28-21,Barclays Bank
3,9013,13350031,93-87-71,Natwest
4,2535,68745993,57-14-32,Barclays Bank


## Removing dashes from sort codes

Within pandas data frames we can still use string methods. In this instance I used the replace method to replace the - with nothing.

In [32]:
transactions["Sort Code"] = transactions["Sort Code"].str.replace("-", "")
transactions.head()

,Transaction ID,Account Number,Sort Code,Bank
0,3888,62230725,959882,Data Source Bank
1,4746,83172326,428638,Barclays Bank
2,5404,34302539,532821,Barclays Bank
3,9013,13350031,938771,Natwest
4,2535,68745993,571432,Barclays Bank


## Combining swift code and transaction data

We can use the pd.merge method to merge the two datasets together. This mege is based on the column which is shared which in this case is the Bank column.

In [33]:
swift = pd.read_csv("C:\\Users\\shani\\PycharmProjects\\preppindata\\Swift Codes.csv")

In [34]:
df = pd.merge(transactions, swift)

In [35]:
df

,Transaction ID,Account Number,Sort Code,Bank,SWIFT code,Check Digits
0,3888,62230725,959882,Data Source Bank,DSBX,12
1,7086,56630552,597744,Data Source Bank,DSBX,12
2,1314,71210735,618579,Data Source Bank,DSBX,12
3,1493,12193988,865442,Data Source Bank,DSBX,12
4,1952,53786334,212754,Data Source Bank,DSBX,12
...,...,...,...,...,...,...
95,6054,47199472,968854,Lloyds Bank,LOYD,C1
96,5773,82021377,438613,Lloyds Bank,LOYD,C1
97,7485,63007765,634617,Lloyds Bank,LOYD,C1
98,8813,83475180,635891,Lloyds Bank,LOYD,C1


## Adding Country Code Column

In [36]:
df.insert(5, "Country Code", "GB")
df

,Transaction ID,Account Number,Sort Code,Bank,SWIFT code,Country Code,Check Digits
0,3888,62230725,959882,Data Source Bank,DSBX,GB,12
1,7086,56630552,597744,Data Source Bank,DSBX,GB,12
2,1314,71210735,618579,Data Source Bank,DSBX,GB,12
3,1493,12193988,865442,Data Source Bank,DSBX,GB,12
4,1952,53786334,212754,Data Source Bank,DSBX,GB,12
...,...,...,...,...,...,...,...
95,6054,47199472,968854,Lloyds Bank,LOYD,GB,C1
96,5773,82021377,438613,Lloyds Bank,LOYD,GB,C1
97,7485,63007765,634617,Lloyds Bank,LOYD,GB,C1
98,8813,83475180,635891,Lloyds Bank,LOYD,GB,C1


## Creating the IBAN Number

The IBAN number is made up of the Country code then check digits then bank code then sort code then account number.

To be able to combine these columns we will first need to check the datatypes of them. If there are strings and numbers present then we will need to convert them all before we can concatenate.

In [37]:
df.dtypes

Transaction ID     int64
Account Number     int64
Sort Code         object
Bank              object
SWIFT code        object
Country Code      object
Check Digits      object
dtype: object

We can see here that the account number will need to be converted before we can join the columns.

In [38]:
df = df.astype('object')
df.dtypes

Transaction ID    object
Account Number    object
Sort Code         object
Bank              object
SWIFT code        object
Country Code      object
Check Digits      object
dtype: object

In [39]:
df['IBAN'] = df['Country Code'] + df['Check Digits'] + df['SWIFT code'] + df['Sort Code'] + df['Account Number']
df

TypeError: can only concatenate str (not "int") to str

In [40]:
df['Account Number'] = df['Account Number'].map(str)

In [41]:
df['IBAN'] = df['Country Code'] + df['Check Digits'] + df['SWIFT code'] + df['Sort Code'] + df['Account Number']
df

,Transaction ID,Account Number,Sort Code,Bank,SWIFT code,Country Code,Check Digits,IBAN
0,3888,62230725,959882,Data Source Bank,DSBX,GB,12,GB12DSBX95988262230725
1,7086,56630552,597744,Data Source Bank,DSBX,GB,12,GB12DSBX59774456630552
2,1314,71210735,618579,Data Source Bank,DSBX,GB,12,GB12DSBX61857971210735
3,1493,12193988,865442,Data Source Bank,DSBX,GB,12,GB12DSBX86544212193988
4,1952,53786334,212754,Data Source Bank,DSBX,GB,12,GB12DSBX21275453786334
...,...,...,...,...,...,...,...,...
95,6054,47199472,968854,Lloyds Bank,LOYD,GB,C1,GBC1LOYD96885447199472
96,5773,82021377,438613,Lloyds Bank,LOYD,GB,C1,GBC1LOYD43861382021377
97,7485,63007765,634617,Lloyds Bank,LOYD,GB,C1,GBC1LOYD63461763007765
98,8813,83475180,635891,Lloyds Bank,LOYD,GB,C1,GBC1LOYD63589183475180


# Final Output

Final stage for the final output was to remove all other columns leaving only the transaction ID's and IBAN numbers

In [42]:
df = df.drop(columns=['Country Code', 'Check Digits', 'SWIFT code', 'Sort Code', 'Account Number', 'Bank'])
df

,Transaction ID,IBAN
0,3888,GB12DSBX95988262230725
1,7086,GB12DSBX59774456630552
2,1314,GB12DSBX61857971210735
3,1493,GB12DSBX86544212193988
4,1952,GB12DSBX21275453786334
...,...,...
95,6054,GBC1LOYD96885447199472
96,5773,GBC1LOYD43861382021377
97,7485,GBC1LOYD63461763007765
98,8813,GBC1LOYD63589183475180
